In [1]:
import tensorflow_compression as tfc

In [2]:
import tensorflow as tf
import glob
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# tf.compat.v1.enable_eager_execution()

In [4]:
import bls2017_modified

In [5]:
!python bls2017_modified.py -h

usage: bls2017_modified.py [-h] [--helpfull] [--verbose]
                           [--num_filters NUM_FILTERS NUM_FILTERS NUM_FILTERS]
                           [--filter_dims FILTER_DIMS FILTER_DIMS FILTER_DIMS]
                           [--conditional CONDITIONAL]
                           [--num_classes NUM_CLASSES]
                           [--checkpoint_dir CHECKPOINT_DIR]
                           {train,compress,decompress} ...

optional arguments:
  -h, --help            show this help message and exit
  --helpfull            show full help message and exit
  --verbose, -V         Report bitrate and distortion when training or
                        compressing. (default: False)
  --num_filters NUM_FILTERS NUM_FILTERS NUM_FILTERS
                        Number of filters per layer; should be 3 integers
                        (default: [64, 64, 64])
  --filter_dims FILTER_DIMS FILTER_DIMS FILTER_DIMS
                        dimensions of filters in the encoding/analysis


In [14]:
!python bls2017_modified.py --verbose train --batchsize=16 --patchsize=32 --last_step=10000 --train_glob='../supervised_compression/data/cifar100png_sub/*'

num_filters [64, 64, 64]
filter_dims [5, 5, 5]
patchsize 32
conditional False
num_classes 1
inside unconditional
<DatasetV1Adapter shapes: (?, 32, 32, 3), types: tf.float32>
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
W1028 12:49:25.731435 4653522368 deprecation.py:323] From bls2017_modified.py:298: DatasetV1.make_one_shot_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Final shape: (?, 32, 32, 3)
data processing is done
INPUTS: (32, 32, 3) False 1 [64, 64, 64] [5, 5, 5]
Inst

INFO:tensorflow:global_step/sec: 21.4209
I1028 12:51:07.139455 4653522368 basic_session_run_hooks.py:692] global_step/sec: 21.4209
INFO:tensorflow:global_step/sec: 23.191
I1028 12:51:11.451415 4653522368 basic_session_run_hooks.py:692] global_step/sec: 23.191
INFO:tensorflow:global_step/sec: 25.9007
I1028 12:51:15.312314 4653522368 basic_session_run_hooks.py:692] global_step/sec: 25.9007
INFO:tensorflow:global_step/sec: 29.1143
I1028 12:51:18.747055 4653522368 basic_session_run_hooks.py:692] global_step/sec: 29.1143
INFO:tensorflow:global_step/sec: 31.0937
I1028 12:51:21.963135 4653522368 basic_session_run_hooks.py:692] global_step/sec: 31.0937
INFO:tensorflow:global_step/sec: 28.0031
I1028 12:51:25.534177 4653522368 basic_session_run_hooks.py:692] global_step/sec: 28.0031
INFO:tensorflow:global_step/sec: 29.4235
I1028 12:51:28.932871 4653522368 basic_session_run_hooks.py:692] global_step/sec: 29.4235
INFO:tensorflow:global_step/sec: 28.7379
I1028 12:51:32.412539 4653522368 basic_sessi

INFO:tensorflow:global_step/sec: 28.432
I1028 12:54:47.977284 4653522368 basic_session_run_hooks.py:692] global_step/sec: 28.432
INFO:tensorflow:global_step/sec: 28.8216
I1028 12:54:51.446906 4653522368 basic_session_run_hooks.py:692] global_step/sec: 28.8216
INFO:tensorflow:global_step/sec: 28.9255
I1028 12:54:54.904062 4653522368 basic_session_run_hooks.py:692] global_step/sec: 28.9255
INFO:tensorflow:global_step/sec: 26.6615
I1028 12:54:58.654788 4653522368 basic_session_run_hooks.py:692] global_step/sec: 26.6615
INFO:tensorflow:global_step/sec: 29.2286
I1028 12:55:02.076100 4653522368 basic_session_run_hooks.py:692] global_step/sec: 29.2286
INFO:tensorflow:global_step/sec: 27.2813
I1028 12:55:05.741621 4653522368 basic_session_run_hooks.py:692] global_step/sec: 27.2813
INFO:tensorflow:global_step/sec: 28.8365
I1028 12:55:09.209451 4653522368 basic_session_run_hooks.py:692] global_step/sec: 28.8365
INFO:tensorflow:global_step/sec: 28.7089
I1028 12:55:12.692690 4653522368 basic_sessi

In [7]:
def read_png(filename):
    """Loads a PNG image file."""
    string = tf.read_file(filename)
    image = tf.image.decode_image(string, channels=3)
    image = tf.cast(image, tf.float32)
    image /= 255
    return image

In [8]:
def crop(image, patchsize):
    return tf.random_crop(image, (patchsize, patchsize, 3))

In [17]:
images = glob.glob('data_sample/*') # glob.glob('../ImageNet-Datasets-Downloader/data/images/*')
patchsize = 32 # 256
num_filters = [64, 64, 64]
filter_dims = [5, 5, 5]
input_size = (patchsize, patchsize, 3)
conditional = False
num_classes = 1 # doesn't matter if conditional = False, is only used for the conditional case
checkpoint_dir ='train/'

In [19]:
# Load checkpoint (trained model) and produce metrics on new images
# Based on the compress function in bls2017.py 
# Currently getting a NotFoundError when loading checkpoint

x = read_png(images[0])
# label = tf.expand_dims(tf.one_hot(int(images[0].split('/')[-1].split('_')[0]), 99), 0)
x = crop(x, 256)
x = tf.expand_dims(x, 0)
# x.set_shape([1, None, None, 3])
x_shape = tf.shape(x)
print(x_shape)

y = bls2017_modified.InferenceNetwork(input_size, conditional, num_classes, num_filters, filter_dims)(x) 
entropy_bottleneck = tfc.EntropyBottleneck()
y_hat, likelihoods = entropy_bottleneck(y, training=False)
x_hat = bls2017_modified.GenerativeNetwork(y_hat.shape[1:], conditional, num_classes, num_filters, filter_dims)(y_hat) #label

num_pixels = tf.cast(tf.reduce_prod(tf.shape(x)[:-1]), dtype=tf.float32)

# Total number of bits divided by number of pixels.
eval_bpp = tf.reduce_sum(tf.log(likelihoods)) / (-np.log(2) * num_pixels)

# Bring both images back to 0..255 range.
x *= 255
x_hat = tf.clip_by_value(x_hat, 0, 1)
x_hat = tf.round(x_hat * 255)

mse = tf.reduce_mean(tf.squared_difference(x, x_hat))
psnr = tf.squeeze(tf.image.psnr(x_hat, x, 255))
msssim = tf.squeeze(tf.image.ssim_multiscale(x_hat, x, 255))

with tf.Session() as sess:
    # Load the latest model checkpoint, get the compressed string and the tensor shapes.
    latest = tf.train.latest_checkpoint(checkpoint_dir=checkpoint_dir)
    print(latest)
    # this keeps producing "Key not found in checkpoint" error
    tf.train.Saver().restore(sess, save_path=latest)

    # Transform the quantized image back and measure performance.
    eval_bpp, mse, psnr, msssim, num_pixels = sess.run(
      [eval_bpp, mse, psnr, msssim, num_pixels])

    # The actual bits per pixel including overhead
    bpp = len(packed.string) * 8 / num_pixels
    print("Mean squared error: {:0.4f}".format(mse))
    print("PSNR (dB): {:0.2f}".format(psnr))
    print("Multiscale SSIM: {:0.4f}".format(msssim))
    print("Multiscale SSIM (dB): {:0.2f}".format(-10 * np.log10(1 - msssim)))
    print("Information content in bpp: {:0.4f}".format(eval_bpp))
    print("Actual bits per pixel: {:0.4f}".format(bpp))

Tensor("Shape:0", shape=(4,), dtype=int32)
INPUTS: (32, 32, 3) False 1 [64, 64, 64] [5, 5, 5]
INPUTS: (32, 32, 64) False 1 [64, 64, 64] [5, 5, 5]
train/model.ckpt-10000
INFO:tensorflow:Restoring parameters from train/model.ckpt-10000


NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key batch_normalization/moving_mean not found in checkpoint
	 [[node save/RestoreV2 (defined at /Users/preethiseshadri/Documents/tf-compression/venv/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'save/RestoreV2':
  File "/Users/preethiseshadri/Documents/tf-compression/venv/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/preethiseshadri/Documents/tf-compression/venv/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/preethiseshadri/Documents/tf-compression/venv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/preethiseshadri/Docum

In [11]:
patchsize = 32 # 256
num_filters = [64, 64, 64]
filter_dims = [5, 5, 5]
input_size = (patchsize, patchsize, 3)
conditional = True
num_classes = 99

output_shape = bls2017_modified.InferenceNetwork(input_size, conditional, num_classes, num_filters, filter_dims).output.shape[1:]
bls2017_modified.InferenceNetwork(input_size, conditional, num_classes, num_filters, filter_dims).summary()

INPUTS: (32, 32, 3) True 99 [64, 64, 64] [5, 5, 5]
INPUTS: (32, 32, 3) True 99 [64, 64, 64] [5, 5, 5]
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 99)]         0                                            
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 16, 16, 64)   4864        encoder_input[0][0]              
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 64)           6400        input_2[0][0]           

In [13]:
bls2017_modified.GenerativeNetwork(output_shape, conditional, num_classes, num_filters, filter_dims).summary()

INPUTS: (4, 4, 64) True 99 [64, 64, 64] [5, 5, 5]
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, 4, 4, 64)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 99)]         0                                            
__________________________________________________________________________________________________
conv2d_transpose_3 (Conv2DTrans (None, 8, 8, 64)     102464      decoder_input[0][0]              
__________________________________________________________________________________________________
dense_12 (Dense)                (None, 64)           6400        input_4[0][0]                    
__________________________________________

In [20]:
import pprint

get_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
var_list = tf.train.list_variables(get_checkpoint)
pprint.pprint(var_list)

[('beta1_power', []),
 ('beta1_power_1', []),
 ('beta2_power', []),
 ('beta2_power_1', []),
 ('bnT_0/beta', [64]),
 ('bnT_0/beta/Adam', [64]),
 ('bnT_0/beta/Adam_1', [64]),
 ('bnT_0/gamma', [64]),
 ('bnT_0/gamma/Adam', [64]),
 ('bnT_0/gamma/Adam_1', [64]),
 ('bnT_0/moving_mean', [64]),
 ('bnT_0/moving_variance', [64]),
 ('bnT_1/beta', [64]),
 ('bnT_1/beta/Adam', [64]),
 ('bnT_1/beta/Adam_1', [64]),
 ('bnT_1/gamma', [64]),
 ('bnT_1/gamma/Adam', [64]),
 ('bnT_1/gamma/Adam_1', [64]),
 ('bnT_1/moving_mean', [64]),
 ('bnT_1/moving_variance', [64]),
 ('bn_0/beta', [64]),
 ('bn_0/beta/Adam', [64]),
 ('bn_0/beta/Adam_1', [64]),
 ('bn_0/gamma', [64]),
 ('bn_0/gamma/Adam', [64]),
 ('bn_0/gamma/Adam_1', [64]),
 ('bn_0/moving_mean', [64]),
 ('bn_0/moving_variance', [64]),
 ('bn_1/beta', [64]),
 ('bn_1/beta/Adam', [64]),
 ('bn_1/beta/Adam_1', [64]),
 ('bn_1/gamma', [64]),
 ('bn_1/gamma/Adam', [64]),
 ('bn_1/gamma/Adam_1', [64]),
 ('bn_1/moving_mean', [64]),
 ('bn_1/moving_variance', [64]),
 ('convT